## 사전준비

### 1. 공통 모듈 import

In [ ]:
import os
import getpass
import uuid
import re
from urllib.parse import urlparse
import http
import json
import time

### 2. API 키 발급 받기

In [ ]:
os.environ["CLOVASTUDIO_API_KEY"] = getpass.getpass("CLOVA Studio API Key: ") # 나중에 청킹화 할 때 필요.

## 문서 전처리하기

### 1. PDF 문서에서 텍스트와 이미지 추출하기 (Load)

이미지 추출이 있을 수는 있음.

In [ ]:
import fitz  # PyMuPDF
import glob
from langchain_core.documents import Document
from pathlib import Path

def extract_documents_from_pdf(pdf_path: str, output_dir: str = "data/extracted_images_문서"):
    os.makedirs(output_dir, exist_ok=True)

    merged_text_path = os.path.join(output_dir, "merged_text" + Path(pdf_path).stem + ".txt")
    merged_text = ""

    doc = fitz.open(pdf_path)
    documents = []

    for i, page in enumerate(doc):
        page_number = i + 1
        page_text = page.get_text("text").strip()
        images_info = []

        # 이미지 추출
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_filename = f"page_{page_number}_img_{img_index+1}.{image_ext}"
            image_path = os.path.join(output_dir, image_filename)

            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)

            images_info.append(image_path)

        # LangChain Document로 변환
        documents.append(Document(
            page_content=page_text,
            metadata={
                "source": os.path.basename(pdf_path),
                "page": page_number,
                "images": ", ".join(images_info)
            }
        ))

        # 병합 텍스트 저장용
        merged_text += f"\n\n--- Page {page_number} ---\n\n{page_text}"

    # 전체 텍스트 저장
    with open(merged_text_path, "w", encoding="utf-8") as f:
        f.write(merged_text)

    return documents, merged_text_path

current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "data")
# print( os.path.join(os.getcwd(), "data"))
pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))

docs = []
print(f"PDF 파일 개수: {len(pdf_files)}")
for pdf_file in pdf_files:
    pdf_path = pdf_file
    temp_docs, merged_path = extract_documents_from_pdf(pdf_file)

    print(f"추출된 문서 페이지 수: {len(temp_docs)}")
    print(f"병합된 텍스트 경로: {merged_path}")
    print(temp_docs[0])  # 하나 확인

    docs.extend(temp_docs)  # 전체 문서 리스트에 추가

In [ ]:
len(docs)

In [ ]:
# 네이버 클라우드에서 발급받은 키를 입력하세요
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("NCP Access Key: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("NCP Secret Key: ")

# 기본 리전 설정
os.environ["AWS_DEFAULT_REGION"] = "kr"

### Convert

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_naver import ChatClovaX

chat_llm = ChatClovaX(
    model="HCX-005"
)

### Chunking

In [ ]:
# -*- coding: utf-8 -*-

class CompletionExecutor:
    def __init__(self, host, api_key, request_id):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'Authorization': self._api_key,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/segmentation', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['topicSeg']
        else:
            print("[CLOVA 응답 오류]", res['status'])
            return 'Error'
        
# file_path = "data/extracted_images_문서/merged_text.txt"

# with open(file_path, "r", encoding="utf-8") as f:
#     text_content = f.read()

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='clovastudio.stream.ntruss.com',
        api_key="Bearer "+os.environ["CLOVASTUDIO_API_KEY"], # 여기 키 형식이 Bearer이 붙네요 
        request_id=str(uuid.uuid4())
    )

    chunked_docs = []

    for doc in docs:  # docs는 페이지별로 추출한 Document 리스트
        segments = completion_executor.execute(
            # 이전 블로그 참고해 파라미터 설정
            {"postProcessMaxSize": 100,   # 후처리 시 하나의 문단이 가질 수 있는 최대 글자 수 (예: 1000자 이하로 잘라줌)
            "alpha": -100,                # 문단 나누기 민감도 조절 파라미터 (기본: 0.0 / -100으로 두면 자동 조정) - 값이 클수록 더 잘게 나뉘고, 작을수록 덜 나뉨
            "segCnt": -1,                 # 원하는 문단 개수 설정 (-1이면 자동 분할, 1 이상의 정수 입력 시 해당 개수로 고정)
            "postProcessMinSize": -1,     # 후처리 시 하나의 문단이 가져야 할 최소 글자 수 (예: 300자 이상 유지)
            "text": doc.page_content,     # 실제 분할할 원본 텍스트
            "postProcess": True}          # 후처리 여부 설정 (True: 문단 길이 균일화 / False: 모델 출력 그대로 사용)
        )

    for seg in segments:
        chunked_docs.append(Document(
            page_content=' '.join(seg),
            metadata=doc.metadata
        ))    

    print(chunked_docs)
    print("chunk 개수 :",len(chunked_docs))

In [ ]:
# image_docs를 chunked_docs에 추가 (원본은 그대로 유지) 
# 원래 image 정보도 같이 받았는데 안 받는 것으로.
combined_docs = chunked_docs

print(f"전체 chunk 개수: {len(combined_docs)}")
print(combined_docs)

In [ ]:
# 샘플 청크 출력
print("\n샘플 청크 (처음 3개):")
for i, chunk in enumerate(combined_docs[:3], 0):
    print(f"\n청크 {i+1}:")
    print(f"내용: {chunk.page_content}")
    print(f"metadata: {chunk.metadata}")
    print(f"길이: {len(chunk.page_content)}자")

### Embedding

In [ ]:
from langchain_naver import ClovaXEmbeddings
 
clovax_embeddings = ClovaXEmbeddings(model='bge-m3') # 임베딩 모델을 설정

text = "임베딩 사용 예제입니다~"
 
clovax_embeddings.embed_query(text)

### Vector Store

In [ ]:
import chromadb
from langchain_chroma import Chroma


# 임베딩 모델 정의
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')

# 로컬 클라이언트 생성
client = chromadb.PersistentClient(path="./Chroma_langchain_db123")

# 컬렉션 준비 (이름 중복 주의!)
collection_name = "clovastudiodatas_docs"
client.get_or_create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}
)

# 벡터스토어 객체 생성
vectorstore_Chroma = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=clovax_embeddings
)

# 문서 추가: 최신 방식은 vectorstore.add_documents 사용
print("Adding documents to Chroma vectorstore...")
for doc in combined_docs:
    try:
        vectorstore_Chroma.add_documents([doc])
        time.sleep(1.1) 
    except Exception as e:
        print(f"[✘] 실패: {doc.metadata} → {e}")

print("All documents have been added to the vectorstore.")


In [ ]:
#FAISS 다운로드
%pip install -qU langchain-community faiss-cpu

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# 임베딩 모델 정의
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')

# FAISS 인덱스 생성 (1024는 bge-m3 차원 수에 맞춰야 함)
index = faiss.IndexFlatIP(1024)  # 내적 기반 검색

# FAISS 벡터스토어 생성
vectorstore_FAISS = FAISS(
    embedding_function=clovax_embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# 문서 일괄 추가 (자동 임베딩 처리)
print("Adding documents to FAISS vectorstore...")
for doc in combined_docs:
    try:
        vectorstore_FAISS.add_documents([doc])
        time.sleep(1.1) 
    except Exception as e:
        print(f"[✘] 실패: {doc.metadata} → {e}")
print("All documents have been added to FAISS vectorstore.")


## 3. 질의하기

### 질문하기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA

# System 및 User 메시지를 나눠 구성
system_template = (
    "당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 원래 가지고있는 지식은 모두 배제하고, 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다."
    "만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요."
)
user_template = (
    "다음은 검색된 문서 내용입니다:\n\n{context}\n\n"
    "위 정보를 바탕으로 다음 질문에 답해주세요:\n{question}"
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template),
])

# 원하는 vectorstore 선택해서 사용
# retriever = vectorstore_Chroma.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={"score_threshold": 0.1, "k": 3}
#     )
retriever = vectorstore_FAISS.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1, "k": 3}
)

# Retrieval QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True
)

# 실행
question = "증빙소득의 연소득 산정방법을 알고 싶어"
result = qa_chain.invoke({"query": question})

print("질문:", question)
print("응답:", result["result"])  # 모델의 실제 응답
for i, doc in enumerate(result["source_documents"]): # 답변시 참고 한 문서
    print(f"\n[출처 문서 {i+1}]\n내용: {doc.page_content}\n메타데이터: {doc.metadata}")